In [2]:
import os, glob
import numpy as np
# import pandas as pd
import tensorflow as tf
from PIL import Image
import matplotlib.pyplot as plt

from keras.models import *
from keras.layers import Input, merge, Conv2D, MaxPooling2D, UpSampling2D, Dropout, Cropping2D, concatenate, Activation,Conv2DTranspose
from keras.layers import BatchNormalization
from keras.optimizers import *
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, History, ReduceLROnPlateau
from keras.callbacks import EarlyStopping
from keras.utils.generic_utils import get_custom_objects

from sklearn.metrics import roc_curve, auc, precision_recall_curve, average_precision_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix

from keras.utils import multi_gpu_model

os.environ["CUDA_VISIBLE_DEVICES"] = "4"

In [3]:
epsilon = K.epsilon()
gamma = 5
alpha = 0.6
beta = 0.6

def recall(y_target, y_pred):
    # clip(t, clip_value_min, clip_value_max) : clip_value_min~clip_value_max 이외 가장자리를 깎아 낸다
    y_target_yn = K.round(K.clip(y_target, 0, 1))
    y_pred_yn = K.round(K.clip(y_pred, 0, 1))
    count_true_positive = K.sum(y_target_yn * y_pred_yn) 
    count_true_positive_false_negative = K.sum(y_target_yn)
    
    recall = count_true_positive / (count_true_positive_false_negative + K.epsilon())
    return recall

def precision(y_target, y_pred):
    y_pred_yn = K.round(K.clip(y_pred, 0, 1))
    y_target_yn = K.round(K.clip(y_target, 0, 1))
    count_true_positive = K.sum(y_target_yn * y_pred_yn) 
    count_true_positive_false_positive = K.sum(y_pred_yn)

    precision = count_true_positive / (count_true_positive_false_positive + K.epsilon())
    return precision

def dice_coef(y_true, y_pred, smooth=0.001):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)

def iou_coef(y_true, y_pred, smooth=1):
    intersection = K.sum(K.abs(y_true * y_pred), axis=[1,2,3])
    union = K.sum(y_true,[1,2,3])+K.sum(y_pred,[1,2,3])-intersection
    iou = K.mean((intersection + smooth) / (union + smooth), axis=0)
    return iou

def dice_coef_loss(y_true, y_pred):
    return 1 - dice_coef(y_true, y_pred)


get_custom_objects().update({
    
    'precision' : precision,
    'recall' : recall,
    'dice_coef' : dice_coef,
    'iou_coef' : iou_coef,
    'dice_coef_loss' : dice_coef_loss,
        
})

img_size = 512
depth = 4
filter_sn = 32

In [5]:
inputs = Input((img_size, img_size,1), dtype='float32')

def encoder_unet(inputs, filters, kernel_size= 3):
    conv = Conv2D(filters, kernel_size, activation=None, padding='same')(inputs)
    conv = BatchNormalization()(conv)
    conv = Activation('relu')(conv)
    conv = Conv2D(filters, kernel_size, activation=None, padding='same')(conv)
    conv = BatchNormalization()(conv)
    conv = Activation('relu')(conv)
#     pool = MaxPooling2D(pool_size=(2, 2))(conv)
    
    return conv

def decoder_unet(inputs, filters, kernel_size= 3, concate_num=depth-1):
    up = concatenate([Conv2DTranspose(filters, (2, 2), strides=(2, 2), padding='same')(inputs), encoders[concate_num]], axis=3)
    conv = Conv2D(filters, kernel_size, activation=None, padding='same')(up)
    conv = BatchNormalization()(conv)
    conv = Activation('relu')(conv)
    conv = Conv2D(filters, kernel_size, activation=None, padding='same')(conv)
    conv = BatchNormalization()(conv)
    conv = Activation('relu')(conv)
    
    return conv

def unet(depth=4, filter_sn=32, img_size=512):
    global encoders
    encoders=[]
    
    inputs = Input((img_size, img_size,1), dtype='float32')
    for down in range(0,depth+1):
        print(down)
        if down == 0:
            encoder_conv = encoder_unet(inputs, filters=filter_sn, kernel_size= 3)
            encoders.append(encoder_conv)
            encoder_conv = MaxPooling2D(pool_size=(2, 2))(encoder_conv)
#             print(filter_sn)
        elif down>0 and down<depth:
            encoder_conv = encoder_unet(encoder_conv, filters=filter_sn*(2**down), kernel_size= 3)
            encoders.append(encoder_conv)
            encoder_conv = MaxPooling2D(pool_size=(2, 2))(encoder_conv)
            print(filter_sn*(2**down))
        else:
            encoder_conv = encoder_unet(encoder_conv, filters=filter_sn*(2**(down-1)), kernel_size= 3)
#             encoders.append(encoder_conv)
            print(filter_sn*(2**(down-1)))
            print(encoder_conv.shape)
    

#         print(filter_sn*(2**depth))
        
    for up in range(0,depth):
        if up == 0:
            decoder_conv= decoder_unet(encoder_conv, filters=filter_sn*(2**(down-1)), kernel_size= 3, concate_num=len(encoders)-1)
        else:
            decoder_conv= decoder_unet(decoder_conv, filters=filter_sn*(2**(depth-up-1)),kernel_size= 3, concate_num=depth-up-1)
        print('decoders:', decoder_conv.shape)
    last_conv = Conv2D(1, (1, 1), activation='sigmoid')(decoder_conv)

    model = Model(inputs=inputs, outputs=last_conv) 
    
    return model

In [6]:
model=unet(depth=4, filter_sn=32, img_size=512)
model.summary()

0
1
64
2
128
3
256
4
256
(?, 32, 32, 256)
decoders: (?, 64, 64, 256)
decoders: (?, 128, 128, 128)
decoders: (?, 256, 256, 64)
decoders: (?, 512, 512, 32)
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 512, 512, 1)  0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 512, 512, 32) 320         input_3[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 512, 512, 32) 128         conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 512, 512, 32) 0

In [ ]:
print("loading data")

train_image = np.load('../data/img200/npy/train_'+str(img_size)+'.npy')
train_label = np.load('../data/img200/npy/label_'+str(img_size)+'.npy')
test_image =np.load('../data/img200/npy/test_'+str(img_size)+'.npy')
# test_label = np.load('../data/npytest/test_label_512.npy')
test_label = np.load('../data/img200/npy/test_label_'+str(img_size)+'.npy')


train_image = train_image/255
train_label = train_label/255
test_image = test_image/255
test_label = test_label/255

print("loading data done")
print(train_image.shape)
print(train_label.shape)
print(test_image.shape)
print(test_label.shape)

In [8]:
model.compile(optimizer=Adam(lr=0.001), loss=dice_coef_loss, metrics=['accuracy', recall, precision, dice_coef])
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, verbose=1, min_delta=1e-4)
earlystopper = EarlyStopping(monitor='val_loss',patience=30, verbose=1)
model_checkpoint = ModelCheckpoint('./result/model_seg_'+str(depth)+'.h5', monitor='val_loss', verbose=1, save_best_only=True, save_weights_only=False)

callbacks_list = [reduce_lr, model_checkpoint, earlystopper]

In [ ]:
results = model.fit(train_image, train_label, batch_size=5, epochs=100, verbose=1,validation_split=0.2, shuffle=True, callbacks=callbacks_list)

In [ ]:
predict = model.predict(test_image, batch_size=5, verbose=1)
np.save('../data/img200/pred/test5_pred_'+str(img_size)+'.npy', pred)

In [ ]:
def preds_Thresh(preds_test,Thresh_value):
    
    preds_mask = np.ndarray((len(test_label),img_size, img_size, 1), dtype=np.float32)
  
    preds_mask0 = preds_test[:,:,:,0] > Thresh_value
  
    preds_mask[:,:,:,0] = preds_mask0*1
    
    return preds_mask

In [ ]:
_loss, _acc, _precision, _recall,_dice_coef = model.evaluate(test_image, test_label, batch_size=5, verbose=1)
print('loss: {:.3f}, accuracy: {:.3f}, precision: {:.3f}, recall: {:.3f}, dice_coef:{:.3f}'.format(_loss, _acc*100, _precision*100, _recall*100, _dice_coef*100))

In [ ]:
preds_mask = preds_Thresh(predict,0.5)
check_len = 5
for toto in range(check_len):
    plt.figure(figsize=(15, 15))
    plt.subplot(2,3,1)
    plt.imshow(test_image[toto,:,:,0],cmap='gray')
    plt.axis('off')
    plt.subplot(2,3,2)
    plt.imshow(test_label[toto,:,:,0],cmap='gray')
    plt.axis('off')
    plt.subplot(2,3,3)
    plt.imshow(test_image[toto,:,:,0],cmap='gray')
    plt.imshow(test_label[toto,:,:,0],cmap='Reds' ,alpha=0.6)
    plt.axis('off')
    plt.subplot(2,3,4)
    plt.imshow(test_image[toto,:,:,0],cmap='gray')
    plt.axis('off')
    plt.subplot(2,3,5)
    plt.imshow(preds_mask[toto,:,:,0],cmap='gray')
    plt.axis('off')
    plt.subplot(2,3,6)
    plt.imshow(test_image[toto,:,:,0],cmap='gray')
    plt.imshow(preds_mask[toto,:,:,0],cmap='Reds' ,alpha=0.6)
    plt.axis('off')

In [ ]:
image_test_label_ravel = np.ravel(test_label) ###image convert to vector
image_test_label_ravel_cur = image_test_label_ravel.astype(np.bool)

predic_ravel_1 = np.ravel(model.predict(test_image, batch_size=10))
predic_ravel_1 = predic_ravel_1.astype(np.float32)



In [ ]:
precision_1, recall_1, thresholds_1 = precision_recall_curve(image_test_label_ravel_cur, predic_ravel_1)
average_precision_score_1 = average_precision_score(image_test_label_ravel_cur, predic_ravel_1)*100

fig = plt.figure(figsize=(10,10))        
plt.title('Precision Recall Curve',fontsize=20)
plt.xlabel('Recall',fontsize=15)
plt.ylabel('Precision',fontsize=15)

plt.plot(recall_1, precision_1, label='Dice_coeff = %0.2f, AP = %0.2f'% (_dice_coef, average_precision_score_1))